# 「テキスト」のエピステモロジー

## BoW(Bag of Words)

In [ ]:
# 15・4・1

# シェークスピアのハムレットとマーロウのフォースタス博士の
# 戯曲テキストを読み込む(出典 : プロジェクト・グーテンベルグ)

import requests

def get_text(url):
    # urlからテキストを読み込む
    r = requests.get(url)
    return r.text

BASE_URL = "https://raw.githubusercontent.com/"
BASE_URL += "shibats/minpy_5th/refs/heads/main/ch15/"
HAMLET_URL = BASE_URL + "hamlet.txt"
DOCTOR_FAUSTUS_URL = BASE_URL + "doctor_faustus.txt"

sha_text = get_text(HAMLET_URL)
mar_text = get_text(DOCTOR_FAUSTUS_URL)

In [ ]:
# 15・4・2
# 戯曲のテキストのうち余分な記号を取り除き，単語に分割
import string

def tokenize(text):
    # 単語リスト作成関数
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))
    return text.split()

sha_words = tokenize(sha_text)
mar_words = tokenize(mar_text)

In [ ]:
# 15・4・3
# counter.Counterを使って，単語ごとのの出現数を数える
from collections import Counter
# 上位単語20件を取得
sha_cnt = Counter(sha_words).most_common(20)
mar_cnt = Counter(mar_words).most_common(20)

In [ ]:
# 15・4・4
# 単語の出現数をグラフに表示
import matplotlib.pyplot as plt

# グラフ表示
s_words, s_counts = zip(*sha_cnt)
m_words, m_counts = zip(*mar_cnt)

fig, axes = plt.subplots(1, 2, figsize=(16, 7))
axes[0].bar(s_words, s_counts, color='skyblue')
axes[0].set_title("Hamlet (Shakespeare) Top 20")

axes[1].bar(m_words, m_counts, color='salmon')
axes[1].set_title("Doctor Faustus (Marlowe) Top 20")

plt.tight_layout()
plt.show()

## N-gram

In [ ]:
# 15・4・5
def generate_ngrams(text, n):
    # 文字列textからN-gramのデータを作り，set型にして返す
    return set(text[i:i+n] for i in range(len(text) - n + 1))

def jaccard_similarity(set1, set2):
    # 2つのN-gramのJaccard係数を返す
    if not set1 or not set2:
        return 0.0
    return len(set1 & set2) / len(set1 | set2)

In [ ]:
# 15・4・6
query = "吾輩は猫である"

corpus = [
    "私は猫である",
    "吾輩は人である",
    "僕は猫なんだよ"
]

gery_2garm = generate_ngrams(query, 2)
for sentence in corpus:
    s_2gram = generate_ngrams(sentence, 2)
    similarity = jaccard_similarity(gery_2garm, s_2gram)
    print(sentence, similarity)


## ベクトル

In [ ]:
# 15・4・7  word2vecのデータをダウンロード(時間がかかります)
!wget "https://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/data/20170201.tar.bz2"

In [ ]:
# 15・4・8  ダウンロードした圧縮ファイルを展開(とても時間がかかります)
!tar jxvf 20170201.tar.bz2

In [ ]:
# 15・4・9  # ライブラリをインストール
!pip install gensim

In [ ]:
# 15・4・10 実行に時間がかかります
import gensim
import numpy as np
# 東北大学の学習済みモデルをロード
model_path = 'entity_vector/entity_vector.model.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

In [ ]:
# 15・4・11  単語を入力してベクトルの最初の10次元，類義語を表示

word = input("日本語の単語を入力してください: ")

# 
if word in model:
    vec = model[word]
    print(f"\n【{word}】のベクトル（最初の10次元）:")
    print(vec[:10])
    print("\n類似語（top5）:")
    print(model.most_similar(word, topn=5))
else:
    print(f"単語「{word}」はモデルに存在しません。")

In [ ]:
# 15・4・12
# 猫と猿，犬，人の類似度を表示

words = ["猿", "犬", "人"]
the_word = "猫"
for word in words:
    similarity = model.similarity(the_word, word)
    print(f"{the_word}と{word}の類似度は{similarity}です。")

# Transformer

In [ ]:
# 15・4・13  必要なライブラリをインストール
!pip install fugashi
!pip install unidic_lite

In [ ]:
# 15・4・14 実行に時間がかかります
import torch
from transformers import BertJapaneseTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

# モデルとトークナイザの読み込み
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")
model = BertModel.from_pretrained("cl-tohoku/bert-base-japanese")

In [ ]:
# 15・4・15
# 文脈中の特定単語のベクトルを取得する関数
def get_token_vector(sentence, target_word):
    tokens = tokenizer.tokenize(sentence)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    token_tensor = torch.tensor([token_ids])

    # 各トークンの位置を調べる（単語が分割される可能性に注意）
    subwords = tokenizer.tokenize(target_word)
    try:
        start_idx = tokens.index(subwords[0])
        end_idx = start_idx + len(subwords)
    except ValueError:
        raise ValueError(f"単語「{target_word}」がトークン列に見つかりませんでした: {tokens}")

    with torch.no_grad():
        output = model(token_tensor)
    embeddings = output.last_hidden_state[0]

    # 対象単語に対応する subword ベクトルの平均を取る
    word_vector = embeddings[start_idx:end_idx].mean(dim=0)
    return word_vector.numpy()

In [ ]:
# 15・4・16
# 類似度計算関数を定義
def cosine_sim(v1, v2):
    return cosine_similarity([v1], [v2])[0][0]

# 文章を受け取り特定の単語の類似度を比較する関数を定義
def compare_sentences(sentence1, sentence2, target_word):
    vec1 = get_token_vector(sentence1, target_word)
    vec2 = get_token_vector(sentence2, target_word)
    return f"文脈中の「{target}」の類似度: {cosine_sim(vec1, vec2):.4f}"

In [ ]:
# 15・4・17
sentence1 = "Pythonはプログラミング言語です"
sentence2 = "Pythonを使ってコードを書きます"
target = "Python"
print(compare_sentences(sentence1, sentence2, target))

In [ ]:
# 15・4・18
# 文と対象単語のペア
sentence1 = "Pythonはプログラミング言語です"
sentence2 = "Monty Pythonは楽しいテレビ番組です"
target = "Python"
print(compare_sentences(sentence1, sentence2, target))